<a href="https://colab.research.google.com/github/samruddhideode/Sanvadita/blob/main/sanvadita_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.naive_bayes import  MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import roc_auc_score, accuracy_score

In [2]:
train_set= pd.read_csv("https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3CubeMahaSent%20Dataset/tweets-train.csv")
test_set= pd.read_csv("https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3CubeMahaSent%20Dataset/tweets-test.csv")
valid_set= pd.read_csv("https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3CubeMahaSent%20Dataset/tweets-valid.csv")

In [7]:
def pre_process(text):
  words = text.split(' ')
  new_words = []
  for word in words:
    if(word == ''): # null strings removed
      continue
    elif(re.search("[a-zA-Z0-9]+", word)): # remove english words and numbers
      continue
    elif(re.search("@[^\s]+", word)): # remove mentions
      continue
    elif(re.search("#[^\s]+", word)): # remove hashtags
      continue
    else:
      word = word.translate(str.maketrans('', '', string.punctuation))
      word = word.replace('\n','')
      new_words.append(word)
  return(new_words)

In [9]:
# convert tweets to a list of constituent words (pre-processing 1) 

train_tweets = []
for tweet in train_set.tweet:
  processed = pre_process(tweet)
  train_tweets.append(processed)

test_tweets = []
for tweet in test_set.tweet:
  processed = pre_process(tweet)
  test_tweets.append(processed)

valid_tweets = []
for tweet in valid_set.tweet:
  processed = pre_process(tweet)
  valid_tweets.append(processed)

print(train_tweets[:10])
print(test_tweets[:10])
print(valid_tweets[:10])


[['ज्येष्ठ', 'पत्रकार', 'अनंत', 'दीक्षित', 'यांच्या', 'निधनाचे', 'वृत्त', 'दुखद', 'आहे', 'चार', 'दशकं', 'त्यांनी', 'आपल्या', 'परखड', 'लेखणीने', 'पत्रकारितेत', 'अमूल्य', 'योगदान', 'दिले', 'दीक्षित', 'यांच्या', 'मार्गदर्शनाखाली', 'पत्रकारांची', 'पिढी', 'घडली', 'अनंत', 'दीक्षित', 'यांना', 'भावपूर्ण', 'श्रद्धांजली', 'आम्ही', 'त्यांच्या', 'परिवाराच्या', 'दुखात', 'सहभागी', 'आहोत'], ['सर्वोच्च', 'न्यायालयाचे', 'निर्देश', 'डावलून', 'पुणे', 'पोलिसांनी', 'प्रा', 'आनंद', 'तेलतुंबडेंना', 'अटक', 'केली', 'मात्र', 'न्यायालयाने', 'त्यांची', 'अटक', 'बेकायदेशीर', 'ठरवून', 'सरकारला', 'जोरदार', 'चपराक', 'लगावली', 'आहे', 'कायदा', 'हातात', 'घेणाया', 'पोलीस', 'अधिकायांवर', 'तात्काळ', 'कारवाई', 'केली'], ['उद्धव', 'ठाकरेंनी', 'भाजपासोबत', 'युती', 'करून', 'शिवसैनिकांचा', 'अवमान', 'व', 'विश्वासघात', 'केला', 'आहे', 'फसवणूक', 'झालेल्या', 'शिवसैनिकांबद्दल', 'मला', 'सहानुभूती', 'असून', 'स्वार्थासाठी', 'उद्धव', 'ठाकरे', 'भाजपसोबत', 'गेले', 'तरी', 'मोदींचे', 'सरकार', 'घालवण्यातच', 'महाराष्ट्राचे', 'हित', 'आहे', 'हे', 

In [24]:
# vectorisation (pre-processing 2)
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)
count = CountVectorizer(tokenizer=identity_tokenizer, lowercase=False)

In [28]:
# use fit_transform() only while TRAINING, otherwise use transform()

vector_tfidf_training = tfidf.fit_transform(train_tweets) #x1_train
vector_count_training = count.fit_transform(train_tweets) #x2_train

vector_tfidf_testing = tfidf.transform(test_tweets) #x1_test
vector_count_testing = count.transform(test_tweets) #x2_test

vector_tfidf_valid = tfidf.transform(valid_tweets) #x1_valid
vector_count_valid = count.transform(valid_tweets) #x2_valid

In [33]:
# model training

# naive-bayes
nb_tfidf = MultinomialNB()
nb_tfidf.fit(vector_tfidf_training,train_set.label)

nb_count = MultinomialNB()
nb_count.fit(vector_count_training,train_set.label)

MultinomialNB()

In [38]:
train_roc = roc_auc_score(train_set.label,nb_tfidf.predict_proba(vector_tfidf_training),multi_class='ovr')
test_roc = roc_auc_score(test_set.label,nb_tfidf.predict_proba(vector_tfidf_testing),multi_class='ovr')
valid_roc = roc_auc_score(valid_set.label,nb_tfidf.predict_proba(vector_tfidf_valid),multi_class='ovr')

print("With tfidf vectorisation: ")
print("train roc_auc: ",train_roc)
print("test roc_auc: ", test_roc)
print("valid roc_auc: ", valid_roc)

train_auc = accuracy_score(train_set.label, nb_tfidf.predict(vector_tfidf_training))
test_auc = accuracy_score(test_set.label,nb_tfidf.predict(vector_tfidf_testing))
valid_auc = accuracy_score(valid_set.label,nb_tfidf.predict(vector_tfidf_valid))

print("\n")
print("train auc: ",train_auc)
print("test auc: ", test_auc)
print("valid auc: ", valid_auc)

With tfidf vectorisation: 
train roc_auc:  0.9749378285354675
test roc_auc:  0.9160311111111111
valid roc_auc:  0.9208173333333334


train auc:  0.8789829948819547
test auc:  0.7622222222222222
valid auc:  0.7693333333333333


In [39]:
train_roc = roc_auc_score(train_set.label,nb_count.predict_proba(vector_count_training),multi_class='ovr')
test_roc = roc_auc_score(test_set.label,nb_count.predict_proba(vector_count_testing),multi_class='ovr')
valid_roc = roc_auc_score(valid_set.label,nb_count.predict_proba(vector_count_valid),multi_class='ovr')

print("With count vectorisation: ")
print("train roc_auc: ",train_roc)
print("test roc_auc: ", test_roc)
print("valid roc_auc: ", valid_roc)

train_auc = accuracy_score(train_set.label, nb_count.predict(vector_count_training))
test_auc = accuracy_score(test_set.label,nb_count.predict(vector_count_testing))
valid_auc = accuracy_score(valid_set.label,nb_count.predict(vector_count_valid))

print("\n")
print("train auc: ",train_auc)
print("test auc: ", test_auc)
print("valid auc: ", valid_auc)

With count vectorisation: 
train roc_auc:  0.9793082502588297
test roc_auc:  0.9152268148148148
valid roc_auc:  0.9193626666666667


train auc:  0.8951626217599472
test auc:  0.7711111111111111
valid auc:  0.7726666666666666
